# DataUt 
## Extracting data from Datainn and puting it to use in Pandas dataframes

In [1]:
# Import of needed libraries

# Libraries for retieving data from the GraphQL API
import urllib3
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Imports for plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams["figure.figsize"] = [14, 7]

import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(14, 7)})
sns.set_context("paper")

# Magic stuff from the Pandas people, flatteing the JSON files into something that can pass off as a table
from pandas.io.json import json_normalize

# Tools to work with dates
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz


# Remove warnings related to HTTPS, no need as data is public
urllib3.disable_warnings()

# And finaly the Pandas library for working with the data 
import pandas as pd

# Fixing filesnames that include wierd chars
from slugify import slugify

# Import calender functions
from calendar import monthrange


In [10]:
exclude_pints =[
"25833V121761",
"74801V2676825",
"66165B885288",
"12135V1126309",
"17764V2679095",
"85865V1125817",
"38973B1816966",
"20927V625367",
"20937V625367",
"80890V625375",
"11462V625377",
"56971V625377",
"37773V625378",
"81732V625378",
"08916V625376",
"22791V625367",
"47140V625367",
"95559B383662",
"41085B1688427",
"89406V384520",
"95599B1820299",
"35294V249020",
"11128V249021",
"51451V2675461",
"15418B1739524",
"11293B1758635",
"18346V886007",
"82149B886007",
"66316V2676855",
"33696V2676853",
"26549V2669516",
"93810V1201961",
"29623V1201961",
"02547B1949899",
"13634B2721359",
"26351B803041",
"07530B1807128",
"44569V248901",
"00381V249612",
"21607V249612",
"65128V249612",
"46684V1842951",
"03908B2426198",
"95507B2418805",
"10437V704575",
"43623V704583",
"41312V705204",
"87256V1213675",
"69326V1060101",
"46288V1060102",
"04219V1060106",
"54236V384508",
"86830B971481",
"88954V971483",
"87740V971816",
"88922B971816",
"19949V971795",
"24927V443403",
"10220V704634",
"60223B1691799",
"63545V180918",
"14340V181285",
"87318V521440",
"09375B1846469",
"28379B1846414",
"80565B1689290",
"96436B1688765",
"76065V521401",
"63839B2077721",
"82726V521407",
"94543V2783687",
"05185V2783689",
"88690B2639744",
"76587V3152664",
"30017V625457",
"51716V625406",
"21255V444277",
"55493V2570030",
"07872V2570034",
"68256V444277",
"07122V443593",
"49146V443594",
"06986V625518",
"21745B1996708",
"66755V625539",
"91158V625541",
"33431V625542",
"22325V625540",
"20680V625530",
"20690V625530",
"40977V625551",
"43746V625550",
"92476B623431",
"66549V625612",
"10254V625614",
"34786V443473",
"71884B1673873",
"09211B1846343",
"65179V1209937",
"66770V1811554",
"18788V1811746",
"63515V1811747",
"14178V705183",
"41542V705184",
"38872V704479",
"09086V1060616",
"19923V1060616",
"26139V1211205",
"57449V2237119",
"13531V1059995",
"93565V2411267",
"32268V1060625",
"48332V1060626",
"89879V1060627",
"24919V1060630",
"89679V72805",
"41300V72150",
"66126V3112188",
"41020V3128088",
"55570V3128090",
"60797V2801059",
"30904V72218",
"93228V578071",
"45215V2055284",
"03614V1740869",
"85036B1824272",
"58182V2395192",
"61777V578597",
"95807V1871351",
"32599V578080",
"81214V578078",
"97043B1797561",
"27756V578605",
"89970V578610",
"91007V578610",
"23077V2604164",
"92019B1685807",
"10239V2725979",
"92743V2726085",
"38681B1802436",
"59217V2725982",
"99040V2725982",
"00911V2725983",
"39643V2725983",
"03711V1060662",
"89276B2803004",
"65758V1796925",
"65345V1796950",
"18959V1796996",
"06154V625229",
"79676V625225",
"26619V625212",
"45026B622830",
"87424V625212",
"91334V625635",
"51368V625246",
"33456V625213",
"19702V625216",
"89094V3154514",
"72406V625599",
"09259V625604",
"60338V625649",
"38023V625278",
"51478V578612",
"92079V1206412",
"67127V885933",
"24290V885934",
"04423V885154",
"96781V885154",
"95133V2486014",
"14354V2486023",
"31675V885942",
"92486V1126283",
"16882V1126285",
"36859V1125764",
"32362V1126287",
"32518V1126287",
"58779V1126296",
"25779V2413305",
"61278B930233",
"98180B930235",
"93618V930601",
"12312V930269",
"76552V72244",
"56025V2078107",
"68733V2078106",
"01124B802238",
"99179B803039",
"03016V805614",
"42954V320585",
"31619V320585",
"93720V3172278",
"17330V930335",
"21796V930339",
"00000V1702751",
"37724V1707378",
"53805V444232",
"20318V625294",
"70429B619257",
"71153B2460281",
"12703B2460285",
"02924V625304",
"30937V444231",
"37055V971362",
"21405V2607269",
"75525V72242",
"12106V72837",
"10719V249533",
"69255V249530",
"14626V249527",
"07156V248908",
"78164V249524",
"39691V249524",
"01521V249521",
"29293V249601",
"70840V2212895",
"86729V384507",
"21514V384489",
"97853V384482",
"66711V384480",
"68385V320580",
"90666V320576",
"22230V121755",
"06390V1204620",
"00870V1204449",
"28020V121748",
"09525V2399484",
"26577V2399484",
"83248B1213727",
"35002V72811",
"25926V2725990",
"62464V2725991",
"25920V885956",
"92394V885954",
"63227V885182",
"18672V578623",
"65182V384013",
"03556V2741763",
"48559V2741763",
"93356V2741763",
"95449V1060638",
"40763V3134118",
"45400V3134151",
"25754V3134152",
"25219V1773688",
"34934V1773715",
"37754V1773669",
"83528V1773686",
"31818V444229",
"69184V2802005",
"76407V1773686",
"30703V180924",
"62963V1060678",
"43286V1060679",
"58042V1060046",
"93714V443439",
"22622V705245",
"20911V121329",
"09091V1204071",
"53409V1204042",
"33412V121301",
"91602V21669",
"42004V22151",
"71445V2809674",
"61942V2809673",
"68462V521378",
"01678V2604140",
"00899B2604165",
"89459V319651",
"34650V319662",
"57430V319659",
"57053V319651",
"49950V319650",
"46694V521392",
"87902V521385",
"12510V521383",
"35551V521381",
"35376V181262",
"89755V180856",
"87557V180903",
"22654V249543",
"10493V249543",
"60539V248942",
"99744V249541",
"32695V249540",
"72532V1060043",
"67576V1060644",
"85039V2725969",
"15060V2726065",
"27620V885957",
"98389V1201959",
"35588V521384",
"11219V22151",
"50741V1751361",
"76138V2038024",
"95081V72156",
"34194V578077",
"83441V578079",
"05420V578059",
"34573V930302",
"05882V3188133",
"01236V705218",
"41838V1060046",
"26773V21770" ]


In [3]:
# Setup of the GQL client to talk to the NPRA Datainn API

sample_transport=RequestsHTTPTransport(
    url='https://www.vegvesen.no/trafikkdata/api/',
    use_json=True,
    headers={
        "Content-type": "application/json",
    },
    verify=False
)

client = Client(
    retries=3,
    transport=sample_transport,
    fetch_schema_from_transport=True,
)

In [4]:
# Function to extract daily volumes from one day to the other

def get_day(startday, stopday, point_id):
    # Dateformat: "2010-01-01T00:00:00+02:00"
    #
    # This functions works slowly - the pagination stuff in GQL is rather code intensive. Som this just brute force
    # the whole idea of pagination by sending a call for every day. If any GQL experts want to fix this please do 
    # and let me know

    query_text_template = '''{trafficData(trafficRegistrationPointId: "{point_id}") {
        volume {
          byDay(from: "{text_page_start}", to: "{text_page_end}") {
            edges {
              node {
                from
                to
                total {
                  volumeNumbers {
                    volume
                  }
                  coverage {
                    percentage
                  }
                }
              }
            }
          }
        }
      }
    }'''


    # Add point ID into template
    query_text_template = query_text_template.replace("{point_id}", point_id)
    
    start= datetime.fromisoformat(startday)
    stop = datetime.fromisoformat(stopday)

    page_start = start
    final_stop = stop

    #Create an empty dataframe with only column names
    df = pd.DataFrame(columns=['from', 'to', 'daily_volume', 'coverage'])

    while page_start < final_stop:
        page_stop = page_start + relativedelta(days=1)

        #Entering start end dates into query text
        query = query_text_template.replace("{text_page_start}", page_start.isoformat(sep='T'))
        query = query.replace("{text_page_end}", page_stop.isoformat(sep='T'))

        #Run query to get data and create temporary dataframe
        Q = gql(query)
        data = client.execute(Q)
        temp_df = json_normalize(data, record_path=['trafficData', 'volume', 'byDay', 'edges'])

        # Fix problem when there is no data
        if len(temp_df.index) > 0:
            column_names = ['from', 'to', 'daily_volume', 'coverage']
            temp_df.columns=column_names
            df = pd.concat([df, temp_df])

        # make move to next page
        page_start = page_stop
    df.set_index('from', inplace = True, drop=False )
    df['csum'] = df['daily_volume'].cumsum()
    return df

In [5]:
def get_day_volme_per_month(year, startmonth, stopmonth, point_id):
    """
    startmonth: integer (1 = Jannuary)
    stopmonth: inteher (12= December)
    """
    # Dateformat: "2010-01-01T00:00:00+02:00"
    #
    # This functions works slowly - the pagination stuff in GQL is rather code intensive. Som this just brute force
    # the whole idea of pagination by sending a call for every day. If any GQL experts want to fix this please do 
    # and let me know

    query_text_template = '''{trafficData(trafficRegistrationPointId: "{point_id}") {
        volume {
          byDay(from: "{text_page_start}", to: "{text_page_end}") {
            edges {
              node {
                from
                to
                total {
                  volumeNumbers {
                    volume
                  }
                  coverage {
                    percentage
                  }
                }
              }
            }
          }
        }
      }
    }'''


    # Add point ID into template
    query_text_template = query_text_template.replace("{point_id}", point_id)
    
    # Create correct timezone for use in queries
    timezone = pytz.timezone("Europe/Oslo")
    
    months = range(startmonth, stopmonth+1)
    
    #Create an empty dataframe with only column names
    df = pd.DataFrame(columns=['from', 'to', 'daily_volume', 'coverage'])
    
    for month in months:
        last_day_of_month = monthrange(year, month)[1]
        
        start  =  datetime(year, month, 1)
        
        # Find last date and solve issue with year roll over
        if (month + 1) == 13:
            end = datetime(year+1, 1, 1)
        else:
            end = datetime(year, month+1, 1)
        
        # Apply correct timezones to date stamps
        start = timezone.localize(start)
        end = timezone.localize(end)
        
        #Entering start end dates into query text
        query = query_text_template.replace("{text_page_start}", start.isoformat(sep='T'))
        query = query.replace("{text_page_end}", end.isoformat(sep='T'))

        #Run query to get data and create temporary dataframe
        Q = gql(query)
        data = client.execute(Q)
        temp_df = json_normalize(data, record_path=['trafficData', 'volume', 'byDay', 'edges'])

        # Fix problem when there is no data
        if len(temp_df.index) > 0:
            column_names = ['from', 'to', 'daily_volume', 'coverage']
            temp_df.columns=column_names
            df = pd.concat([df, temp_df])
        
        
    df.set_index('from', inplace = True, drop=False )
    df['csum'] = df['daily_volume'].cumsum()
    return df
        
aaa = get_day_volme_per_month(2019, 12, 12, "35002V72811")

aaa.head(33)
    
    
    


,from,to,daily_volume,coverage,csum
from,,,,,
2019-12-01T00:00:00+01:00,2019-12-01T00:00:00+01:00,2019-12-02T00:00:00+01:00,24661,75.00,24661
2019-12-02T00:00:00+01:00,2019-12-02T00:00:00+01:00,2019-12-03T00:00:00+01:00,42048,75.00,66709
2019-12-03T00:00:00+01:00,2019-12-03T00:00:00+01:00,2019-12-04T00:00:00+01:00,40108,75.00,106817
2019-12-04T00:00:00+01:00,2019-12-04T00:00:00+01:00,2019-12-05T00:00:00+01:00,39674,75.00,146491
2019-12-05T00:00:00+01:00,2019-12-05T00:00:00+01:00,2019-12-06T00:00:00+01:00,41216,75.00,187707
2019-12-06T00:00:00+01:00,2019-12-06T00:00:00+01:00,2019-12-07T00:00:00+01:00,43243,75.00,230950
2019-12-07T00:00:00+01:00,2019-12-07T00:00:00+01:00,2019-12-08T00:00:00+01:00,25459,75.00,256409
2019-12-08T00:00:00+01:00,2019-12-08T00:00:00+01:00,2019-12-09T00:00:00+01:00,23037,75.00,279446
2019-12-09T00:00:00+01:00,2019-12-09T00:00:00+01:00,2019-12-10T00:00:00+01:00,44790,75.00,324236


In [11]:

def plott_anual_comparison_to_pdf(startmonth, endmonth, point, point_name, filename):
    '''
    startmonth: integer
    endmonth: integer
    example point id: "35002V72811"
    '''
    #timezone = pytz.timezone("Europe/Oslo")
    #startdate = timezone.localize(startdate)
    #enddate = timezone.localize(enddate)
            
    #start2019 = startdate - pd.DateOffset(years=1)
    #end2019 = enddate - pd.DateOffset(years=1)
        
    volume_2020 = get_day_volme_per_month(2020, startmonth, endmonth, point)
    volume_2019 = get_day_volme_per_month(2019, startmonth, endmonth, point)
    
    #volume_2020 = get_day(startdate.isoformat(sep='T'), enddate.isoformat(sep='T'), point)
    #volume_2019 = get_day(start2019.isoformat(sep='T'), end2019.isoformat(sep='T'), point)
    
    #Shifting the 2019 data one year forward and plotting
    volume_2020['fra'] = pd.to_datetime(volume_2020['from'])
    volume_2020.set_index('fra', inplace = True, drop=False )

    volume_2019['fra'] = pd.to_datetime(volume_2019['from']) + pd.DateOffset(years=1)
    volume_2019.set_index('fra', inplace = True, drop=False )
    
    # Plot setup
    plt.figure(figsize=(11.7,8.3))
    
    # Plotting of the recieved data
    plt.plot(volume_2020['csum'], label='2020 Data')
    plt.plot(volume_2019['csum'], label='2019 Data')

    # Adding a legend
    title = point_name + '  -  ' + point
       
    plt.suptitle(title)
    plt.title( "Statens vegvesen - Transportutvikling, Transportstyring")
    
    plt.xlabel('Dato')
    plt.ylabel("Total antall kjøretøy")
    plt.legend()
    
    filename = slugify(filename)
    f_name = './results/' + filename + '.pdf'
    plt.savefig(f_name)
    plt.close()


In [7]:
# Generate a testplott
plott_anual_comparison_to_pdf(1, 12, "35002V72811", "Trondheim", "a")

/home/tlev/anaconda3/lib/python3.7/site-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


In [8]:
# Restore list of vehicle counting point from Vehicle_Counting_point notebook

%store -r vehicle_cointing_points
vehicle_cointing_points.head()

,id,name,lat,lon,road_reference,road_category,type,county,municipality
0,56100V804816,RAMPE SØRÅS MOT HOP,60.325803,5.337152,EV39 S78D1 m6434 KD3 m172,Europaveg,VEHICLE,Vestland,Bergen
1,03486V319647,ØLEN/ETNE,59.623114,5.886108,EV134 S4D1 m17825,Europaveg,VEHICLE,Rogaland,Vindafjord
2,20318V625294,FESTNINGTUNNEL,59.911180,10.724291,EV18 S55D1 m5076,Europaveg,VEHICLE,Oslo,Oslo
3,00030V705190,MATRAND S,60.020618,12.113421,RV2 S2D1 m4510,Riksveg,VEHICLE,Innlandet,Eidskog
4,29614V805708,SOTRABRUA VEST,60.372022,5.155573,RV555 S1D1 m11739,Riksveg,VEHICLE,Vestland,Øygarden


In [12]:
start_month =  1
end_month=   5

for i in range(len(vehicle_cointing_points)): 
    
    
    id =vehicle_cointing_points.loc[i, "id"]
    if not id in exclude_pints and not 'B' in id:
        name = vehicle_cointing_points.loc[i, "name"]
        road_reference = vehicle_cointing_points.loc[i, "road_reference"]
        
        title = name + '\n' + road_reference 
        file_name = road_reference + '' + name

        print(id + '  -  ' + name ) 
        try:
            plott_anual_comparison_to_pdf(start_month, end_month, id, title, file_name)
        except:
            print("Data extraction failed for point: {}".format(id))
    else:
        print("{} was on the exclude list and therefore removed".format(id))

56100V804816  -  RAMPE SØRÅS MOT HOP
03486V319647  -  ØLEN/ETNE
20318V625294 was on the exclude list and therefore removed
00030V705190  -  MATRAND S
29614V805708  -  SOTRABRUA VEST
02087V625292  -  EV 18 V/ MASTEMYR
62464V2725991 was on the exclude list and therefore removed
76092V705200  -  ALVDAL
Data extraction failed for point: 76092V705200
31504V578608  -  SNÅSAHEIA
Data extraction failed for point: 31504V578608
42004V22151 was on the exclude list and therefore removed
70741V384496  -  NAUSTDALSTUNNELEN
05960V384024  -  REED VEST
63227V885182 was on the exclude list and therefore removed
37055V971362 was on the exclude list and therefore removed
10239V2725979 was on the exclude list and therefore removed
58272V2583663  -  Granstunnelen sørgående
21565V1060643  -  JORA
27436V2172082  -  HANEKLEIV SYD
80565B1689290 was on the exclude list and therefore removed
22500V805651  -  HOVLAND
73380V121327  -  Rampe mot Prestheia
Data extraction failed for point: 73380V121327
53155V1665289 

Data extraction failed for point: 97351V930305
41300V72150 was on the exclude list and therefore removed
80777V121763  -  KILE
82149B886007 was on the exclude list and therefore removed
85320V885198  -  NAPPSTRAUMEN TUNNEL
20808V2037771  -  BOGSRUD PÅRAMPE
61523V971803  -  ST HANSFJELLET
59199V72809  -  MELHUS SØR
26461V971783  -  LØKKEBERG
71535V319524  -  Lassa
03364V971801  -  STRØMSHAUG
88980V805612  -  MEHAMMAR
10229V1060056  -  KALLERUD SØR
06938V521177  -  Kjørbekk
Data extraction failed for point: 06938V521177
76361V1665570  -  SØRDALSTUNNEL (BS24)
04512V930297  -  VEDBOTN
Data extraction failed for point: 04512V930297
61723V2435179  -  STORSKOG
Data extraction failed for point: 61723V2435179
94975V2411852  -  Rostad
25046V805611  -  STUDALEN FARTSTAVLE
33419V930316  -  Vardø Sentrum
Data extraction failed for point: 33419V930316
58345V1719147  -  EIDETTUNNELEN NORD
38203V2433139  -  BAGN SØR
20697V805009  -  NØSTET - PUD.BRU
31475V885919  -  SELFORS
48559V2741763 was on the ex

20895V805602  -  DALSLEID VEST
71989V885955  -  SVOLVÆR Ø.
48498V1125765  -  HEGGELIA
88184V2303031  -  Eikeberget SV rampe
20680V625530 was on the exclude list and therefore removed
33731V249521  -  Sætre
86830B971481 was on the exclude list and therefore removed
60371V1719219  -  SOLBERG NORD
49530V804761  -  TRENGEREID-OSTERØY
Data extraction failed for point: 49530V804761
57808V2282277  -  KLEVERUD
90978V1126285  -  Setermoen Sør
Data extraction failed for point: 90978V1126285
76881V578085  -  Verdal arm Verdalsøra-Levanger
94845V521171  -  Ørviksletta
00899B2604165 was on the exclude list and therefore removed
78492V2394249  -  Rotvollekra
12478V320582  -  JÅTTEN
63025V885155  -  HERGOTTA
83717V1126309  -  HARSTAD BYSKILLET (K
42324V1060631  -  BEGNADALEN
73691B1835999 was on the exclude list and therefore removed
73894V444221  -  JESSHEIM EV6
91821V930281  -  E6 Kirkenes X FV367 Øst
13606V2303025  -  Eikeberget SØ rampe
84212V805616  -  EIDSVÅGTUNNELEN
10493V249543 was on the exc

74031V1857387  -  LYSAKERLOKKET RAMPE
Data extraction failed for point: 74031V1857387
63839B2077721 was on the exclude list and therefore removed
65484V930593  -  TRANSFARELV
26266V443149  -  E6 NYE MOSS NORD
94013V521403  -  Søve
66883V930281  -  SOLHEIMSLIA
67055V804829  -  DANMPLASS-VESTRE
05120V625649  -  E6 Alnabru Nordgående Helsfyr Furuset
Data extraction failed for point: 05120V625649
95559B383662 was on the exclude list and therefore removed
16868V805119  -  LAGUNEN-TROLDHAUGTUN. SØRÅS
00902V578622  -  Hegra øst
Data extraction failed for point: 00902V578622
68234V443152  -  TARALDRUD
14061V443595  -  RV159 VAREMESSA AVRAMPE
00509V885112  -  NORDLANDSPORTEN
Data extraction failed for point: 00509V885112
19085V885118  -  FELLINGFORS
Data extraction failed for point: 19085V885118
87318V521440 was on the exclude list and therefore removed
56025V2078107 was on the exclude list and therefore removed
46321V444232  -  RAMSTADSLETTA
Data extraction failed for point: 46321V444232
14340

86675V1060056  -  KALLERUD NORD
39653V2172095  -  HELLAND SYD
35573V2037768  -  NEBBENES PÅRAMPE
20586V2721334  -  Flyplassvegen Sola PÅKJ mot sola
11293B1758635 was on the exclude list and therefore removed
11786V2441492  -  ARM BØMLO-SVEIO
95509V319514  -  Søylandskiosken
72861V2491863  -  E6 Skibotn x E8
Data extraction failed for point: 72861V2491863
31157V2782679  -  Tromsøysundtunnelen T2
21607V249612 was on the exclude list and therefore removed
60303V930233  -  E6 Altaveien (ved sykkelpunkt)
99353V625530  -  Ø.Akersvei 1,2
57449V2237119 was on the exclude list and therefore removed
20911V121329 was on the exclude list and therefore removed
51449V885379  -  SOMMERSET/Dragsarmen
35511V805647  -  LÅTEFOSS
82885V805617  -  FLØYFJ SØRGÅENDE
62685V885925  -  RØKLAND NORD
99179B803039 was on the exclude list and therefore removed
62931V705208  -  KIRKENÆR N
63354V971485  -  GAUTESTAD
04219V1060106 was on the exclude list and therefore removed
26549V2669516 was on the exclude list and 

33412V121301 was on the exclude list and therefore removed
14204V805611  -  E39 VABAKKEN
77465V384509  -  TOTLAND
62500V2791531  -  Granfoss rampe vest 1
58420V805630  -  SELJESTAD
Data extraction failed for point: 58420V805630
44161V2411262  -  HUNDORPTUNNELEN
66549V625612 was on the exclude list and therefore removed
23077V2604164 was on the exclude list and therefore removed
43746V625550 was on the exclude list and therefore removed
66126V3112188 was on the exclude list and therefore removed
64438V2370744  -  HARDANGERBRUA
95807V1871351 was on the exclude list and therefore removed
24919V1060630 was on the exclude list and therefore removed
21796V930339 was on the exclude list and therefore removed
14178V705183 was on the exclude list and therefore removed
49769V443454  -  OSL EV16 GARDERMOEN MED
Data extraction failed for point: 49769V443454
70840V2212895 was on the exclude list and therefore removed
72958V805656  -  SVELGANE
32842V319521  -  Mosheim
39608V521433  -  Spjotsodd
1259

73809V805608  -  SVEIO
63845V2659017  -  MELLOMDOKKA RASTEPLASS RAMPE
Data extraction failed for point: 63845V2659017
99467V578621  -  Hegra vest
Data extraction failed for point: 99467V578621
97860V249525  -  Håhjem
13433V319582  -  FORUS ved stv.Aftenbladet Avkj fra Sandnes
82177V625405  -  Gaustad 1,2,3,4,5
22231V320583  -  Finnestad
17188V930317  -  Vadsø Kai
81103V1059995  -  FÅVANG SØR
Data extraction failed for point: 81103V1059995
21008V885200  -  E10 Ramberg
61777V578597 was on the exclude list and therefore removed
48271V2370824  -  BU ØST
53324V181283  -  KVERK
Data extraction failed for point: 53324V181283
78251V1719132  -  BJØRNSTAD
69255V249530 was on the exclude list and therefore removed
34947V319593  -  Ev39 Forus nord Avkjøring mot Forus
Data extraction failed for point: 34947V319593
56355V2788140  -  NYELV
71657V971306  -  JONSTEN ØST
Data extraction failed for point: 71657V971306
73840V2041694  -  Østre Tangent
39028V1201951  -  FLÅ SYD
66410V625367  -  AMMERUD
Data

In [ ]:
column_names = ['from', 'to', 'daily_volume', 'coverage', 'point_type', 'county', 'municipality']

df2020 = pd.DataFrame(columns = column_names)
df2019 = pd.DataFrame(columns = column_names)

for i in range(len(vehicle_cointing_points)): 
    id =vehicle_cointing_points.loc[i, "id"]
    
    point_type  = vehicle_cointing_points.loc[i, "type"]
    
    if not (id in exclude_pints) and (point_type == 'VEHICLE'):
        print("Working on point {}".format(id))
        name = vehicle_cointing_points.loc[i, "name"]
        road_reference = vehicle_cointing_points.loc[i, "road_reference"]
        county = vehicle_cointing_points.loc[i, "county"]
        municipality = vehicle_cointing_points.loc[i, "municipality"]
        
        try: 
            volume_2020 = get_day_volme_per_month(2020, 1, 4, id)
            volume_2019 = get_day_volme_per_month(2019, 1, 4, id)

            volume_2020['point_type'] = point_type
            volume_2020['county'] = county
            volume_2020['municipality'] = municipality   

            volume_2019['point_type'] = point_type
            volume_2019['county'] = county
            volume_2019['municipality'] = municipality

            if len(volume_2020.index) > 0:
                df2020 = pd.concat([df2020, volume_2020])
            if len(volume_2019.index) > 0:
                df2019 = pd.concat([df2019, volume_2019])
        except:
            print("Failed for traffic count point {}".format(id))


Working on point 56100V804816


/home/tlev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/tlev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Working on point 03486V319647
Working on point 00030V705190
Working on point 29614V805708
Working on point 02087V625292
Working on point 76092V705200
Failed for traffic count point 76092V705200
Working on point 31504V578608
Failed for traffic count point 31504V578608
Working on point 70741V384496
Working on point 05960V384024
Working on point 58272V2583663
Working on point 21565V1060643
Working on point 27436V2172082
Working on point 22500V805651
Working on point 73380V121327
Failed for traffic count point 73380V121327
Working on point 53155V1665289
Failed for traffic count point 53155V1665289
Working on point 97687V72808
Working on point 27227V885156
Working on point 72741V1811592
Working on point 78805V2347241
Working on point 21381V2554057
Working on point 14551V705197
Working on point 71241V2460301
Working on point 18523V2282344
Failed for traffic count point 18523V2282344
Working on point 19690V804744
Working on point 69193V248908
Working on point 85039V2725969
Failed for traffic 